# Performance Evaluation in Cell Lines

## Contents

- [Data Loading](#data-loading)
- [Drug-level Pearson Correlation](#drug-level-pearson-correlation)
- [Drug-Level Pearson Correlation Stratified by Biological Mechanism](#drug-level-pearson-correlation-stratified-by-biological-mechanism)
- [Drug-Level Pearson Correlation Stratified by Tissue Type](#drug-level-pearson-correlation-stratified-by-tissue-type)
- [auROC Analysis](#auroc-analysis)
- [Response Rate Analysis](#response-rate-analysis)

In [ ]:
from __future__ import annotations

import json

import altair as alt
import pandas as pd
import numpy as np
import typing as t
import sklearn.metrics as skm

from pathlib import Path
from scipy import stats

from cdrpy.datasets import Dataset
from cdrpy.data.preprocess import GroupStandardScaler

from screendl.utils import evaluation as eval_utils

from utils.eval import ResponseRateEvaluator, auroc
from utils.plot import MODEL_COLORS, DEFAULT_BOXPLOT_CONFIG, configure_chart

## Data Loading

In [ ]:
root = Path("../../../datastore")

In [ ]:
drug_types_path = root / "processed/DrugAnnotations/drug_types.json"
fixed_drug_types = {"chemo": "Chemo", "targeted": "Targeted", "other": "Other"}
with open(drug_types_path, "r") as fh:
    drug_to_type = {k: fixed_drug_types[v] for k,v in json.load(fh).items()}

In [ ]:
dataset_dir = root / "inputs/CellModelPassports-GDSCv1v2"

cell_meta = pd.read_csv(dataset_dir / "MetaCellAnnotations.csv", index_col=0)
drug_meta = pd.read_csv(dataset_dir / "MetaDrugAnnotations.csv", index_col=0)
drug_meta["type"] = drug_meta.index.map(drug_to_type)

D = Dataset.from_csv(
    dataset_dir / "LabelsLogIC50.csv",
    cell_meta=cell_meta,
    drug_meta=drug_meta,
    name=dataset_dir.name,
)

print(D)

In [ ]:
def load_multirun_predictions(
    multirun_dir: str | Path, regex: str, splits: list[str] | None = None
) -> pd.DataFrame:
    """Loads predictions from a multirun."""
    if isinstance(multirun_dir, str):
        multirun_dir = Path(multirun_dir)

    file_list = multirun_dir.glob(regex)
    pred_df = pd.concat(map(pd.read_csv, file_list))

    if splits is not None:
        pred_df = pred_df[pred_df["split_group"].isin(splits)]

    return pred_df

In [ ]:
def rescale_predictions(df: pd.DataFrame) -> pd.DataFrame:
    """Rescales the predictions based on predictions in the train set."""
    df_trn = df[df["split_group"] == "train"]
    df_tst = df[df["split_group"] == "test"]

    gss = GroupStandardScaler()
    df_trn["y_true"] = gss.fit_transform(df_trn[["y_true"]], groups=df_trn["drug_id"])
    df_tst["y_true"] = gss.transform(df_tst[["y_true"]], groups=df_tst["drug_id"])

    gss = GroupStandardScaler()
    df_trn["y_pred"] = gss.fit_transform(df_trn[["y_pred"]], groups=df_trn["drug_id"])
    df_tst["y_pred"] = gss.transform(df_tst[["y_pred"]], groups=df_tst["drug_id"])

    return pd.concat([df_trn, df_tst])

In [ ]:
model_results: t.Dict[str, pd.DataFrame] = {}
output_dir = root / "outputs"
path_fmt = "basic/{0}/{1}/multiruns/{2}"
column_mapper = {"fold": "split_id", "split": "split_group"}

In [ ]:
# HiDRA results

model = "HiDRA-legacy"
date = "2024-04-17_19-29-28"

run_dir = output_dir / path_fmt.format(D.name, model, date)
run_regex = "*/predictions.csv"

model_results[model.split("-")[0]] = (
    load_multirun_predictions(run_dir, run_regex, splits=["train", "test"])
    .rename(columns=column_mapper)
    .groupby("split_id", as_index=False)
    .apply(rescale_predictions)
    .assign(model=model.split("-")[0])
)

In [ ]:
# DualGCN results

model = "DualGCN-legacy"
dates = [
    # NOTE: split over multiple runs
    "2024-04-12_09-31-07",
    "2024-04-12_09-32-20",
    "2024-04-14_08-02-47",
    "2024-04-14_08-03-56",
    "2024-04-15_16-17-18",
]

temp = []
for date in dates:
    run_dir = output_dir / path_fmt.format(D.name, model, date)
    run_regex = "*/predictions.csv"
    temp.append(load_multirun_predictions(run_dir, run_regex, splits=["train", "test"]))

model_results[model.split("-")[0]] = (
    pd.concat(temp)
    .rename(columns=column_mapper)
    .groupby("split_id", as_index=False)
    .apply(rescale_predictions)
    .assign(model=model.split("-")[0])
)

In [ ]:
model = "DeepCDR-legacy"
date = "2024-04-02_09-27-37"

run_dir = output_dir / path_fmt.format(D.name, model, date)
run_regex = "*/predictions.csv"

model_results[model.split("-")[0]] = (
    load_multirun_predictions(run_dir, run_regex, splits=["train", "test"])
    .rename(columns=column_mapper)
    .groupby("split_id", as_index=False)
    .apply(rescale_predictions)
    .assign(model=model.split("-")[0])
)

In [ ]:
model = "ScreenDL"
date = "2025-04-23_10-33-36"
path_fmt = "screenahead/{0}/{1}/multiruns/{2}"

run_dir = output_dir / path_fmt.format(D.name, model, date)

model_results[model + "-PT"] = (
    load_multirun_predictions(run_dir, "*/predictions.csv", splits=["train", "test"])
    .groupby("split_id", as_index=False)
    .apply(rescale_predictions)
    .assign(model=model + "-PT")
)

temp_ = model_results[model + "-PT"].query("split_group == 'train'").copy()
model_results[model + "-SA (ALL)"] = (
    load_multirun_predictions(run_dir, "*/predictions_sa.csv", splits=None)
    .pipe(lambda df: pd.concat([df, temp_]))
    .groupby("split_id", as_index=False)
    .apply(rescale_predictions)
    .assign(model=model + "-SA (ALL)")
)

In [ ]:
model_results_df = pd.concat(model_results.values())
model_results_df.head()

In [ ]:
_temp_nbs = (
    model_results_df.query("model == 'ScreenDL-SA (ALL)'")
    .fillna({"was_screened": False})
    .query("was_screened == False")
    .assign(model="ScreenDL-SA")
    .copy()
)

model_results_df = pd.concat(
    [model_results_df.query("model != 'ScreenDL-SA (ALL)'"), _temp_nbs]
)

## Baseline Models

In [ ]:
# # load baselines
path_fmt = "experiments/baselines/{0}/multiruns/{1}"
dataset = "CellModelPassports-GDSCv1v2"
run_dates = [
    ("2025-06-30_10-44-42", "Ridge"),
    ("2025-06-30_10-43-31", "Random Forest"),
]

baseline_results = []
for run_date, name in run_dates:
    run_dir = output_dir / path_fmt.format(dataset, run_date)
    run_preds = load_multirun_predictions(run_dir, "*/predictions.csv")
    baseline_results.append(run_preds.assign(model=name, date=run_date))

baseline_results = pd.concat(baseline_results).dropna(subset=["y_true", "y_pred"])
baseline_results.head()

In [ ]:
model_results_df_trn = model_results_df.query("split_group == 'train'")
model_results_df_tst = model_results_df.query("split_group == 'test'")

In [ ]:
model_results_df_tst = pd.concat([model_results_df_tst, baseline_results])
model_results_df_tst = model_results_df_tst.fillna({"was_screened": False})

## Drug-level Pearson Correlation

In [ ]:
MODELS = [
    "DualGCN",
    "HiDRA",
    "DeepCDR",
    "Ridge",
    "Random Forest",
    "ScreenDL-PT",
    "ScreenDL-SA",
]

In [ ]:
pcc_metrics = (
    model_results_df_tst.groupby(["model", "drug_id"])
    .apply(lambda g: eval_utils.pcorr(g, "y_true", "y_pred"))
    .to_frame(name="pcc")
    .reset_index()
)

pcc_metrics.groupby("model")["pcc"].describe().loc[MODELS]

In [ ]:
# compare performance of ScreenDL-SA with ScreenDL-PT
pcc_metrics_wide = pcc_metrics.set_index(["drug_id", "model"])["pcc"].unstack()

m1, m2 = "ScreenDL-PT", "ScreenDL-SA"
stats.wilcoxon(pcc_metrics_wide[m1], pcc_metrics_wide[m2])

In [ ]:
# compare performance of ScreenDL-PT with DeepCDR

m1, m2 = "DeepCDR", "ScreenDL-PT"
stats.wilcoxon(pcc_metrics_wide[m1], pcc_metrics_wide[m2])

In [ ]:
model_to_color = {k: v for k, v in MODEL_COLORS.items() if k in MODELS}
MODEL_COLOR_SCALE = alt.Scale(
    domain=list(model_to_color.keys()), range=list(model_to_color.values())
)

In [ ]:
annots = D.drug_meta[["target_pathway", "type"]].reset_index()
source = pcc_metrics.merge(annots, on="drug_id")

In [ ]:
# number of drugs per class (chemo vs targeted vs other)
source.drop_duplicates("drug_id")["type"].value_counts(dropna=False)

In [ ]:
boxes_all_drugs = (
    alt.Chart(source, width=37 * len(MODELS), height=380)
    .mark_boxplot(**DEFAULT_BOXPLOT_CONFIG)
    .encode(
        alt.X("model:N")
        .axis(labelAngle=-45, labelPadding=5)
        .sort(MODELS)
        .title(None),
        alt.Y("pcc:Q")
        .axis(titlePadding=10, tickCount=6, grid=False)
        .scale(domain=[-0.2, 1])
        .title("Pearson Correlation"),
        alt.Color("model:N", scale=MODEL_COLOR_SCALE).legend(None),
    )
)

boxes_drug_types = (
    alt.Chart(source, width=37 * 2, height=380)
    .transform_filter(alt.FieldOneOfPredicate("type", ["Targeted", "Chemo"]))
    .mark_boxplot(**DEFAULT_BOXPLOT_CONFIG)
    .encode(
        alt.Column("model:N")
        .spacing(10)
        .sort(MODELS)
        .title(None)
        .header(orient="top"),
        alt.X("type:N")
        .axis(labelAngle=-45, labelPadding=5, orient="bottom")
        .sort(["Targeted", "Chemo"])
        .title(None),
        alt.Y("pcc:Q")
        .axis(grid=False, labels=False, ticks=False, domainOpacity=0)
        .scale(domain=[-0.2, 1])
        .title(None),
        alt.Color("model:N", scale=MODEL_COLOR_SCALE).legend(None),
    )
)

pcc_boxplot = alt.hconcat(boxes_all_drugs, boxes_drug_types, spacing=5)
configure_chart(pcc_boxplot)

## Drug-Level Pearson Correlation Stratified by Biological Mechanism

In [ ]:
MIN_DRUGS_PER_PATHWAY = 2

drug_ids = pcc_metrics["drug_id"].unique()
drug_to_pathway = D.drug_meta.loc[drug_ids]["target_pathway"]

drugs_per_pathway = drug_meta["target_pathway"].value_counts()
keep_pathways = drugs_per_pathway[drugs_per_pathway >= MIN_DRUGS_PER_PATHWAY].index
drug_to_pathway = drug_to_pathway.map(lambda x: x if x in keep_pathways else "Other")
drug_to_pathway = drug_to_pathway.map(lambda x: x if x != "Unclassified" else "Other")
drug_to_pathway.value_counts()

In [ ]:
pathway_pcc_metrics = (
    pcc_metrics.assign(target_pathway=lambda df: df["drug_id"].map(drug_to_pathway))
    .groupby(["model", "target_pathway"])
    .agg({"pcc": "median", "drug_id": "nunique"})
    .reset_index()
    .rename(columns={"drug_id": "count", "pcc": "median_pcc"})
)
pathway_pcc_metrics.head()

In [ ]:
best_model_by_pathway = (
    pathway_pcc_metrics[~pathway_pcc_metrics["model"].str.contains("ScreenDL-SA")]
    .groupby("target_pathway", as_index=False)
    .apply(lambda g: g.loc[g["median_pcc"].idxmax()])
)

best_model_by_pathway["model"].value_counts() / len(best_model_by_pathway)

In [ ]:
sorted_pathways = (
    pathway_pcc_metrics.query("model == 'ScreenDL-PT'")
    .sort_values("median_pcc", ascending=False)["target_pathway"]
    .to_list()
)

points = (
    alt.Chart(pathway_pcc_metrics, width=250, height=380)
    .mark_circle()
    .encode(
        alt.X("median_pcc:Q")
        .axis(titlePadding=10, values=[0.0, 0.2, 0.4, 0.6, 0.8], grid=False)
        .scale(domain=(0.0, 0.8))
        .title("Median Pearson Correlation Per Drug"),
        alt.Y("target_pathway:N").sort(sorted_pathways).title(None),
        alt.Color("model:N", scale=MODEL_COLOR_SCALE).legend(
            orient="top", title=None, symbolStrokeWidth=1, columns=4
        ),
        tooltip=["median_pcc:Q", "target_pathway:N"],
    )
)

bars = alt.Chart(
    pathway_pcc_metrics.query("model == 'ScreenDL-PT'"), width=60, height=380
).encode(
    alt.X("count:Q")
    .axis(grid=False, values=[0, 40, 80], titlePadding=10)
    .scale(domain=(0, 80))
    .title("No. Drugs"),
    alt.Y("target_pathway:N")
    .axis(ticks=False, labels=False, offset=0, domainOpacity=0)
    .sort(sorted_pathways)
    .title(None),
    text="count",
)

bars = bars.mark_bar() + bars.mark_text(align="left", dx=4, fontSize=10)

pathway_pcc_chart = alt.hconcat(points, bars, spacing=10)

(
    configure_chart(pathway_pcc_chart)
    .configure_circle(
        size=100,
        opacity=0.8,
        fillOpacity=1,
        strokeOpacity=1,
        stroke="black",
        strokeWidth=0.5,
    )
    .configure_bar(stroke="black", strokeWidth=0.5, size=15, color="#999999")
    .configure_legend(columns=4)
)

## Drug-Level Pearson Correlation Stratified by Tissue Type

In [ ]:
MIN_TUMORS_PER_TISSUE = 5
tumor_to_tissue = D.cell_meta["tissue"]

tissue_pcc_metrics = (
    model_results_df_tst.assign(tissue=lambda df: df["cell_id"].map(tumor_to_tissue))
    .groupby(["model", "tissue", "drug_id"])
    .apply(eval_utils.pcorr)
    .groupby(["model", "tissue"])
    .median()
    .dropna()
    .to_frame(name="median_pcc")
    .reset_index()
    .assign(count=lambda df: df["tissue"].map(tumor_to_tissue.value_counts()))
    .assign(
        tissue=lambda df: df.apply(
            lambda r: r["tissue"] if r["count"] >= MIN_TUMORS_PER_TISSUE else "Other",
            axis=1,
        )
    )
)
tissue_pcc_metrics.head()

In [ ]:
best_model_by_tissue = (
    tissue_pcc_metrics[~tissue_pcc_metrics["model"].str.contains("ScreenDL-SA")]
    .groupby("tissue", as_index=False)
    .apply(lambda g: g.loc[g["median_pcc"].idxmax()])
)

best_model_by_tissue["model"].value_counts() / len(best_model_by_tissue)

In [ ]:
sorted_tissues = (
    tissue_pcc_metrics.query("model == 'ScreenDL-PT'")
    .sort_values("median_pcc", ascending=False)["tissue"]
    .to_list()
)

points = (
    alt.Chart(tissue_pcc_metrics, width=250, height=290)
    .mark_circle()
    .encode(
        alt.X("median_pcc:Q")
        .axis(titlePadding=10, values=[-0.2, 0.0, 0.2, 0.4, 0.6, 0.8], grid=False)
        .scale(domain=(-0.2, 0.8))
        .title("Median Pearson Correlation Per Drug"),
        alt.Y("tissue:N").sort(sorted_tissues).title(None),
        alt.Color("model:N", scale=MODEL_COLOR_SCALE).legend(
            orient="top", title=None, symbolStrokeWidth=1, columns=4
        ),
    )
)

bars = alt.Chart(
    tissue_pcc_metrics.query("model == 'ScreenDL-PT'"), width=60, height=290
).encode(
    alt.X("count:Q")
    .axis(grid=False, values=[0, 100, 200], titlePadding=10)
    .scale(domain=(0, 200))
    .title("No. Cell Lines"),
    alt.Y("tissue:N")
    .axis(ticks=False, labels=False, offset=0, domainOpacity=0)
    .sort(sorted_tissues)
    .title(None),
    text="count",
)

bars = bars.mark_bar() + bars.mark_text(align="left", dx=4, fontSize=10)
tissue_pcc_chart = alt.hconcat(points, bars, spacing=10)

(
    configure_chart(tissue_pcc_chart)
    .configure_circle(
        size=100,
        opacity=0.8,
        fillOpacity=1,
        strokeOpacity=1,
        stroke="black",
        strokeWidth=0.5,
    )
    .configure_bar(stroke="black", strokeWidth=0.5, size=15.5, color="#999999")
)

## auROC Analysis

In [ ]:
def get_responder_assignments(df: pd.DataFrame, q: float) -> pd.DataFrame:
    df_train = df[df["split_group"] == "train"]
    df_test = df[df["split_group"] == "test"]

    thresh = df_train["y_true"].quantile(q)

    df_test["y_true_class"] = (df_test["y_true"] < thresh).astype(int)

    return df_test


get_response_rate = lambda df: df["y_true_class"].sum() / len(df["y_true_class"])
is_best_index = lambda g: g.index == g.idxmin()

In [ ]:
# annotate responders vs non-responders and select best drugs for each model

TRUE_RESPONDER_PERCENTILE = 0.3

model_results_df_responder_cls = (
    model_results_df.groupby(["model", "split_id", "drug_id"], as_index=False)
    .apply(lambda g: get_responder_assignments(g, TRUE_RESPONDER_PERCENTILE))
    .reset_index(drop=True)
    .assign(
        selected_drug=lambda df: df.groupby(["model", "cell_id"])["y_pred"].transform(
            is_best_index
        )
    )
)

model_results_df_responder_cls.head()

In [ ]:
y_true_df_ = (
    model_results_df_responder_cls.query("model == 'ScreenDL-PT'")
    .filter(items=["cell_id", "drug_id", "y_true_class"])
    .drop_duplicates()
)

In [ ]:
baseline_results_responder_cls: pd.DataFrame = (
    baseline_results.merge(y_true_df_, on=["cell_id", "drug_id"])
    .assign(
        selected_drug=lambda df: df.groupby(["model", "cell_id"])["y_pred"].transform(
            is_best_index
        )
    )
    .reset_index(drop=True)
)

baseline_results_responder_cls.head()

In [ ]:
drug_auroc_metrics = (
    pd.concat([model_results_df_responder_cls, baseline_results_responder_cls])
    .groupby(["model", "drug_id"])
    .apply(auroc, col1="y_true_class", col2="y_pred")
    .to_frame(name="auROC")
    .reset_index()
)

drug_auroc_metrics.groupby("model")["auROC"].describe().loc[MODELS]

In [ ]:
# compare performance of ScreenDL-SA with ScreenDL-PT
drug_auroc_metrics_wide = drug_auroc_metrics.set_index(["drug_id", "model"])
drug_auroc_metrics_wide = drug_auroc_metrics_wide["auROC"].unstack()

m1, m2 = "ScreenDL-PT", "ScreenDL-SA"
stats.wilcoxon(drug_auroc_metrics_wide[m1], drug_auroc_metrics_wide[m2])

In [ ]:
sorted_models = (
    drug_auroc_metrics.groupby("model")["auROC"]
    .median()
    .sort_values(ascending=False)
    .index.to_list()
)

bars = (
    alt.Chart()
    .mark_bar(stroke="black", strokeWidth=1, fillOpacity=1, strokeOpacity=1)
    .encode(
        alt.X("median(auROC):Q")
        .axis(grid=False, tickCount=5, domainColor="black", titlePadding=10)
        .scale(domain=(0.4, 1.0))
        .title("auROC"),
        alt.Y("model:N")
        .axis(domainColor="black")
        .scale(domain=sorted_models, paddingOuter=0.15)
        .title(None),
        alt.Color("model:N", scale=MODEL_COLOR_SCALE).legend(None),
    )
    .properties(width=260, height=300)
)

error_bars = (
    alt.Chart()
    .mark_errorbar(
        extent="iqr", ticks=alt.MarkConfig(size=5, color="black", strokeWidth=1)
    )
    .encode(alt.Y("model:N"), alt.X("auROC:Q"))
)

rule = (
    alt.Chart(pd.DataFrame({"x": [0.5]}))
    .mark_rule(stroke="black", strokeDash=[4, 3], strokeWidth=1.25)
    .encode(x="x:Q")
)

drug_auroc_chart = alt.layer(bars, error_bars, rule, data=drug_auroc_metrics)
configure_chart(drug_auroc_chart)

## Response Rate Analysis

In [ ]:
rre = ResponseRateEvaluator(
    y_pred_var="y_pred",
    y_true_var="y_true_class",
    n_iter=10,
)

rr_metrics = (
    pd.concat([model_results_df_responder_cls, baseline_results_responder_cls])
    .groupby(["model"])
    .apply(rre.eval)
    .to_frame(name="rr")
    .reset_index()
)

rr_metrics

In [ ]:
pt_selected_drugs = model_results_df_responder_cls.query(
    "model == 'ScreenDL-PT' and selected_drug == True"
)
no_uniq_drugs = pt_selected_drugs["drug_id"].nunique()
no_uniq_pathways = (
    pt_selected_drugs["drug_id"].map(D.drug_meta["target_pathway"]).nunique()
)
print(f"No. Drugs (ScreenDL-PT): {no_uniq_drugs}")
print(f"No. Pathways (ScreenDL-PT): {no_uniq_pathways}")

In [ ]:
sa_selected_drugs = model_results_df_responder_cls.query(
    "model == 'ScreenDL-SA' and selected_drug == True"
)
no_uniq_drugs = sa_selected_drugs["drug_id"].nunique()
no_uniq_pathways = (
    sa_selected_drugs["drug_id"].map(D.drug_meta["target_pathway"]).nunique()
)
print(f"No. Drugs (ScreenDL-PT): {no_uniq_drugs}")
print(f"No. Pathways (ScreenDL-PT): {no_uniq_pathways}")

In [ ]:
random_response_rates = []
for _ in range(1000):
    selected_random = y_true_df_.groupby("cell_id").sample(1)
    response_rate = selected_random["y_true_class"].sum() / len(selected_random)
    random_response_rates.append(response_rate)

random_response_rate = np.mean(random_response_rates)
assert np.isclose(random_response_rate, TRUE_RESPONDER_PERCENTILE, atol=0.01)
print(f"Random Response Rate {random_response_rate:.2f}")

In [ ]:
sorted_models = rr_metrics.sort_values("rr", ascending=False)["model"].to_list()

base = alt.Chart(rr_metrics)

bars = (
    base.mark_bar(stroke="black", strokeWidth=1, fillOpacity=1, strokeOpacity=1)
    .encode(
        alt.X("rr:Q")
        .axis(grid=False, tickCount=5, domainColor="black", format="%", titlePadding=10)
        .scale(domain=(0.0, 1.0))
        .title("Response Rate (%)"),
        alt.Y("model:N")
        .axis(domainColor="black")
        .scale(domain=sorted_models, paddingOuter=0.15)
        .title(None),
        alt.Color("model:N", scale=MODEL_COLOR_SCALE).legend(None),
    )
    .properties(width=260, height=300)
)

text = base.mark_text(align="left", dx=6, fontSize=10).encode(
    alt.X("rr:Q")
    .axis(grid=False, tickCount=5, domainColor="black", format="%", titlePadding=10)
    .scale(domain=(0.0, 1.0))
    .title("Response Rate (%)"),
    alt.Y("model:N")
    .axis(domainColor="black")
    .scale(domain=sorted_models, paddingOuter=0.15)
    .title(None),
    alt.Text("rr:Q", format=".1%"),
)

rule = (
    alt.Chart(pd.DataFrame({"x": [random_response_rate]}))
    .mark_rule(stroke="black", strokeDash=[4, 3], strokeWidth=1.25)
    .encode(x="x:Q")
)

rr_chart = alt.layer(bars, text, rule)
configure_chart(rr_chart)

In [ ]:
top = alt.hconcat(pcc_boxplot, pathway_pcc_chart, spacing=50)
top = top.resolve_scale(color="independent")

bot = alt.hconcat(tissue_pcc_chart, drug_auroc_chart, rr_chart, spacing=60)
bot = bot.resolve_scale(color="independent")

final_chart = alt.vconcat(top, bot, spacing=35).resolve_scale(color="independent")

(
    configure_chart(final_chart)
    .configure_circle(size=100, opacity=0.8, stroke="black", strokeWidth=0.5)
    .configure_bar(stroke="black", strokeWidth=0.5, color="#999999")
)